In [36]:
!pip install rasterio

In [37]:
!pip install geopandas

In [38]:
import os
import numpy as np
import pandas as pd
import rasterio
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing

In [39]:
# Định nghĩa các thông số
BASE_PATH = "/kaggle/input/btl-ai/DATA_SV"
HIMA_PATH = os.path.join(BASE_PATH, "Hima")
ERA5_PATH = os.path.join(BASE_PATH, "ERA5")
PRECIP_PATH = os.path.join(BASE_PATH, "Precipitation/Radar")
OUTPUT_PATH = "/kaggle/working/output/"
SHAPEFILE_PATH = "/kaggle/input/shapefileprovince/gadm41_VNM_1.shp"
os.makedirs(OUTPUT_PATH, exist_ok=True)

HIMA_BANDS = ['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B', 'I2B', 'I4B', 'IRB', 'VSB', 'WVB']
ERA5_PARAMS = ['CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850']
SELECTED_HIMA_BANDS = ['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B', 'I2B', 'I4B', 'IRB', 'VSB', 'WVB']
SELECTED_ERA5_PARAMS = ['CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850']
SELECTED_FEATURES = SELECTED_HIMA_BANDS + SELECTED_ERA5_PARAMS
HEIGHT, WIDTH = 90, 250

In [40]:
# Đọc shapefile và lọc tỉnh Nghệ An
gdf = gpd.read_file(SHAPEFILE_PATH)
gdf = gdf.to_crs(epsg=4326)
gdf_nghean = gdf[gdf['NAME_1'] == 'Nghệ An']
if gdf_nghean.empty:
    raise ValueError("Không tìm thấy tỉnh Nghệ An trong shapefile!")

In [41]:
# Hàm kiểm tra pixel trong tỉnh
def filter_pixels_in_province(df, gdf_province, feature_name):
    try:
        geometry = [Point(x, y) for x, y in zip(df['x'], df['y'])]
        gdf_pixels = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        gdf_within = gpd.sjoin(gdf_pixels, gdf_province, predicate='within', how='left')
        filtered_df = gdf_within.dropna(subset=['index_right'])
        return filtered_df[['x', 'y', feature_name]]
    except Exception as e:
        print(f"Lỗi khi lọc pixel trong tỉnh cho {feature_name}: {e}")
        return pd.DataFrame()

In [42]:
# Hàm trích xuất đặc trưng
def extract_features(file_path, data_type):
    try:
        with rasterio.open(file_path) as src:
            data = src.read(1)
            transform = src.transform
            height, width = src.height, src.width
            nodata = src.nodata

            feature_name = os.path.basename(file_path).split('_')[0]
            rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
            x_coords, y_coords = rasterio.transform.xy(transform, rows.flatten(), cols.flatten())
            values = data.flatten()

            if nodata is not None:
                values = np.where(values == nodata, np.nan, values)
            values = np.where(values == -9999, np.nan, values)
            if data_type == "Radar":
                valid_mask = np.isfinite(values)
                values = np.where(valid_mask & (values < 0), np.nan, values)
            elif data_type == "Hima" and feature_name in ['B04B', 'B05B', 'B06B', 'VSB']:
                dt = parse_datetime_from_filename(os.path.basename(file_path), "Hima")
                is_daytime = 6 <= dt.hour < 18
                if not is_daytime:
                    values = np.nan

            valid_mask = np.isfinite(values)
            df = pd.DataFrame({
                'x': np.array(x_coords)[valid_mask],
                'y': np.array(y_coords)[valid_mask],
                feature_name: values[valid_mask]
            })
            print(f"Trích xuất {feature_name} từ {file_path}: {len(df)} pixel hợp lệ")
            return df
    except Exception as e:
        print(f"Lỗi khi trích xuất đặc trưng từ {file_path}: {e}")
        return None

In [43]:
# Hàm phân tích thời gian từ tên file
def parse_datetime_from_filename(filename, data_type):
    try:
        if data_type == "Hima":
            parts = filename.split('_')
            if len(parts) < 2:
                return None
            time_part = parts[1].split('_TB.tif')[0]
            time_part = time_part.replace('.Z', '')
            dt = datetime.strptime(time_part, '%Y%m%d%H%M')
        elif data_type == "ERA5":
            parts = filename.split('_')
            if len(parts) < 2:
                return None
            time_part = parts[1].replace('.tif', '')
            dt = datetime.strptime(time_part, '%Y%m%d%H%M%S')
        elif data_type == "Radar":
            time_part = filename.split('_')[1].replace('.tif', '')
            dt = datetime.strptime(time_part, '%Y%m%d%H%M%S')
        else:
            return None
        return dt.replace(minute=0, second=0, microsecond=0)
    except Exception as e:
        global error_count
        if error_count < 5:
            print(f"Error parsing datetime from {filename} (type {data_type}): {e}")
            error_count += 1
        return None
error_count = 0

In [44]:
# Hàm thu thập file
def collect_files(base_path, data_type, expected_subdirs=None):
    files_dict = {}
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith('.tif'):
                file_path = os.path.join(root, file)
                dt = parse_datetime_from_filename(file, data_type)
                if dt is None:
                    continue
                if expected_subdirs:
                    subdir = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(file_path)))))
                    if dt not in files_dict:
                        files_dict[dt] = {}
                    files_dict[dt][subdir] = file_path
                else:
                    files_dict[dt] = file_path
    return files_dict

In [45]:
# Hàm tạo bảng dữ liệu cho một thời điểm
def create_data_table_for_dt(dt, hima_files, era5_files, radar_file, gdf_province):
    try:
        # Kiểm tra xem có đủ file không
        for band in SELECTED_HIMA_BANDS:
            if band not in hima_files or not os.path.exists(hima_files[band]):
                print(f"Thiếu file Himawari cho {band} tại {dt}")
                return None
        for param in SELECTED_ERA5_PARAMS:
            if param not in era5_files or not os.path.exists(era5_files[param]):
                print(f"Thiếu file ERA5 cho {param} tại {dt}")
                return None
        if not os.path.exists(radar_file):
            print(f"Thiếu file Radar tại {dt}")
            return None

        # Trích xuất và lọc pixel trong tỉnh
        valid_pixels = None
        hima_dfs = []
        for band in SELECTED_HIMA_BANDS:
            df = extract_features(hima_files[band], "Hima")
            if df is None or df.empty:
                print(f"Không có dữ liệu cho Himawari {band} tại {dt}")
                return None
            if valid_pixels is None:
                df = filter_pixels_in_province(df, gdf_province, band)
                if df.empty:
                    print(f"Không có pixel hợp lệ cho Himawari tại {dt} trong Nghệ An")
                    return None
                valid_pixels = set(zip(df['x'], df['y']))
            else:
                df = df[df[['x', 'y']].apply(tuple, axis=1).isin(valid_pixels)]
            hima_dfs.append(df)

        hima_df = hima_dfs[0]
        for df in hima_dfs[1:]:
            hima_df = hima_df.merge(df, on=['x', 'y'], how='inner')
            hima_df = hima_df.dropna()

        era5_dfs = []
        for param in SELECTED_ERA5_PARAMS:
            df = extract_features(era5_files[param], "ERA5")
            if df is None or df.empty:
                print(f"Không có dữ liệu cho ERA5 {param} tại {dt}")
                return None
            df = df[df[['x', 'y']].apply(tuple, axis=1).isin(valid_pixels)]
            era5_dfs.append(df)

        era5_df = era5_dfs[0]
        for df in era5_dfs[1:]:
            era5_df = era5_df.merge(df, on=['x', 'y'], how='inner')
            era5_df = era5_df.dropna()

        radar_df = extract_features(radar_file, "Radar")
        if radar_df is None or radar_df.empty:
            print(f"Không có dữ liệu cho Radar tại {dt}")
            return None
        radar_df = radar_df[radar_df[['x', 'y']].apply(tuple, axis=1).isin(valid_pixels)]
        if radar_df.empty:
            print(f"Không có pixel hợp lệ cho Radar tại {dt} trong Nghệ An")
            return None

        merged_df = hima_df.merge(era5_df, on=['x', 'y'], how='inner')
        merged_df = merged_df.merge(radar_df, on=['x', 'y'], how='inner')
        merged_df = merged_df.dropna()
        merged_df['datetime'] = dt
        print(f"Tạo thành công DataFrame cho {dt} với {len(merged_df)} hàng")
        return merged_df
    except Exception as e:
        print(f"Lỗi khi xử lý {dt}: {e}")
        return None

In [46]:
# Thu thập dữ liệu
print("Collecting Himawari files...")
hima_files_dict = {}
for band in SELECTED_HIMA_BANDS:
    band_path = os.path.join(HIMA_PATH, band)
    band_files = collect_files(band_path, "Hima")
    for dt, path in band_files.items():
        if dt not in hima_files_dict:
            hima_files_dict[dt] = {}
        hima_files_dict[dt][band] = path

print("Collecting ERA5 files...")
era5_files_dict = {}
for param in SELECTED_ERA5_PARAMS:
    param_path = os.path.join(ERA5_PATH, param)
    param_files = collect_files(param_path, "ERA5")
    for dt, path in param_files.items():
        if dt not in era5_files_dict:
            era5_files_dict[dt] = {}
        era5_files_dict[dt][param] = path

print("Collecting Radar files...")
radar_files_dict = collect_files(PRECIP_PATH, "Radar")

In [47]:
# Đồng bộ thời gian và lọc thời điểm hợp lệ
common_datetimes = set(hima_files_dict.keys()) & set(era5_files_dict.keys()) & set(radar_files_dict.keys())
common_datetimes = sorted(list(common_datetimes))
print(f"Số thời điểm đồng bộ: {len(common_datetimes)}")

Số thời điểm đồng bộ: 2337


In [48]:
valid_datetimes = []
for dt in common_datetimes:
    hima_files = hima_files_dict.get(dt, {})
    era5_files = era5_files_dict.get(dt, {})
    radar_file = radar_files_dict.get(dt)
    missing = False
    for band in SELECTED_HIMA_BANDS:
        if band not in hima_files or not os.path.exists(hima_files[band]):
            print(f"Bỏ qua {dt} do thiếu file Himawari {band}")
            missing = True
            break
    for param in SELECTED_ERA5_PARAMS:
        if param not in era5_files or not os.path.exists(era5_files[param]):
            print(f"Bỏ qua {dt} do thiếu file ERA5 {param}")
            missing = True
            break
    if not radar_file or not os.path.exists(radar_file):
        print(f"Bỏ qua {dt} do thiếu file Radar")
        missing = True
    if not missing:
        valid_datetimes.append(dt)

print(f"Số thời điểm hợp lệ sau khi lọc: {len(valid_datetimes)}")

Bỏ qua 2019-04-01 06:00:00 do thiếu file Himawari I4B
Bỏ qua 2019-04-01 07:00:00 do thiếu file Himawari I4B
Bỏ qua 2019-04-01 12:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 13:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 14:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 16:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 17:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 18:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 19:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 20:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 21:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 22:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-01 23:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-02 05:00:00 do thiếu file Himawari I4B
Bỏ qua 2019-04-02 06:00:00 do thiếu file Himawari I4B
Bỏ qua 2019-04-02 07:00:00 do thiếu file Himawari I4B
Bỏ qua 2019-04-02 11:00:00 do thiếu file Himawari B06B
Bỏ qua 2019-04-02 12:00:00 do thiếu file Himawari B04B
Bỏ qua 2019-04-

In [49]:
# Tạo bảng dữ liệu (chạy tuần tự)
all_dataframes = []
for dt in valid_datetimes:
    hima_files = hima_files_dict.get(dt, {})
    era5_files = era5_files_dict.get(dt, {})
    radar_file = radar_files_dict.get(dt)
    df = create_data_table_for_dt(dt, hima_files, era5_files, radar_file, gdf_nghean)
    if df is not None:
        all_dataframes.append(df)

Lỗi khi trích xuất đặc trưng từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0000_TB.tif: 'float' object is not subscriptable
Không có dữ liệu cho Himawari B04B tại 2019-04-01 00:00:00
Lỗi khi trích xuất đặc trưng từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0100_TB.tif: 'float' object is not subscriptable
Không có dữ liệu cho Himawari B04B tại 2019-04-01 01:00:00
Lỗi khi trích xuất đặc trưng từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0200_TB.tif: 'float' object is not subscriptable
Không có dữ liệu cho Himawari B04B tại 2019-04-01 02:00:00
Lỗi khi trích xuất đặc trưng từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0300_TB.tif: 'float' object is not subscriptable
Không có dữ liệu cho Himawari B04B tại 2019-04-01 03:00:00
Lỗi khi trích xuất đặc trưng từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0400_TB.tif: 'float' object is not subscriptable
Không có dữ liệu cho Himawari B04B

/tmp/ipykernel_31/3026505838.py:20: RuntimeWarning: invalid value encountered in less
  values = np.where(valid_mask & (values < 0), np.nan, values)


Tạo thành công DataFrame cho 2019-04-01 08:00:00 với 889 hàng
Trích xuất B04B từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B05B từ /kaggle/input/btl-ai/DATA_SV/Hima/B05B/2019/04/01/B05B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B06B từ /kaggle/input/btl-ai/DATA_SV/Hima/B06B/2019/04/01/B06B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B09B từ /kaggle/input/btl-ai/DATA_SV/Hima/B09B/2019/04/01/B09B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B10B từ /kaggle/input/btl-ai/DATA_SV/Hima/B10B/2019/04/01/B10B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B11B từ /kaggle/input/btl-ai/DATA_SV/Hima/B11B/2019/04/01/B11B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B12B từ /kaggle/input/btl-ai/DATA_SV/Hima/B12B/2019/04/01/B12B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B14B từ /kaggle/input/btl-ai/DATA_SV/Hima/B14B/2019/04/01/B14B_20190401.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất

/tmp/ipykernel_31/3026505838.py:20: RuntimeWarning: invalid value encountered in less
  values = np.where(valid_mask & (values < 0), np.nan, values)


Tạo thành công DataFrame cho 2019-04-01 09:00:00 với 889 hàng
Trích xuất B04B từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2019/04/01/B04B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B05B từ /kaggle/input/btl-ai/DATA_SV/Hima/B05B/2019/04/01/B05B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B06B từ /kaggle/input/btl-ai/DATA_SV/Hima/B06B/2019/04/01/B06B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B09B từ /kaggle/input/btl-ai/DATA_SV/Hima/B09B/2019/04/01/B09B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B10B từ /kaggle/input/btl-ai/DATA_SV/Hima/B10B/2019/04/01/B10B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B11B từ /kaggle/input/btl-ai/DATA_SV/Hima/B11B/2019/04/01/B11B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B12B từ /kaggle/input/btl-ai/DATA_SV/Hima/B12B/2019/04/01/B12B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B14B từ /kaggle/input/btl-ai/DATA_SV/Hima/B14B/2019/04/01/B14B_20190401.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất

...

/tmp/ipykernel_31/3026505838.py:20: RuntimeWarning: invalid value encountered in less
  values = np.where(valid_mask & (values < 0), np.nan, values)


Tạo thành công DataFrame cho 2020-10-31 08:00:00 với 889 hàng
Trích xuất B04B từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2020/10/31/B04B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B05B từ /kaggle/input/btl-ai/DATA_SV/Hima/B05B/2020/10/31/B05B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B06B từ /kaggle/input/btl-ai/DATA_SV/Hima/B06B/2020/10/31/B06B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B09B từ /kaggle/input/btl-ai/DATA_SV/Hima/B09B/2020/10/31/B09B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B10B từ /kaggle/input/btl-ai/DATA_SV/Hima/B10B/2020/10/31/B10B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B11B từ /kaggle/input/btl-ai/DATA_SV/Hima/B11B/2020/10/31/B11B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B12B từ /kaggle/input/btl-ai/DATA_SV/Hima/B12B/2020/10/31/B12B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất B14B từ /kaggle/input/btl-ai/DATA_SV/Hima/B14B/2020/10/31/B14B_20201031.Z0900_TB.tif: 22500 pixel hợp lệ
Trích xuất

/tmp/ipykernel_31/3026505838.py:20: RuntimeWarning: invalid value encountered in less
  values = np.where(valid_mask & (values < 0), np.nan, values)


Tạo thành công DataFrame cho 2020-10-31 09:00:00 với 889 hàng
Trích xuất B04B từ /kaggle/input/btl-ai/DATA_SV/Hima/B04B/2020/10/31/B04B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B05B từ /kaggle/input/btl-ai/DATA_SV/Hima/B05B/2020/10/31/B05B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B06B từ /kaggle/input/btl-ai/DATA_SV/Hima/B06B/2020/10/31/B06B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B09B từ /kaggle/input/btl-ai/DATA_SV/Hima/B09B/2020/10/31/B09B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B10B từ /kaggle/input/btl-ai/DATA_SV/Hima/B10B/2020/10/31/B10B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B11B từ /kaggle/input/btl-ai/DATA_SV/Hima/B11B/2020/10/31/B11B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B12B từ /kaggle/input/btl-ai/DATA_SV/Hima/B12B/2020/10/31/B12B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất B14B từ /kaggle/input/btl-ai/DATA_SV/Hima/B14B/2020/10/31/B14B_20201031.Z1000_TB.tif: 22500 pixel hợp lệ
Trích xuất

/tmp/ipykernel_31/3026505838.py:20: RuntimeWarning: invalid value encountered in less
  values = np.where(valid_mask & (values < 0), np.nan, values)


In [50]:
# Kết hợp và lưu dữ liệu
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    final_df = final_df.dropna()
    keep_columns = ['x', 'y'] + SELECTED_FEATURES + ['Radar', 'datetime']
    final_df = final_df[keep_columns]
    print(f"Kích thước bảng dữ liệu cuối cùng: {len(final_df)} hàng")
    final_df.to_csv(os.path.join(OUTPUT_PATH, "weather_data_nghean.csv"), index=False)
else:
    raise ValueError("Không thể tạo bảng dữ liệu do không có thời điểm hợp lệ.")

Kích thước bảng dữ liệu cuối cùng: 388493 hàng


In [51]:
# Chuẩn bị dữ liệu huấn luyện
X = final_df[SELECTED_FEATURES]
y = final_df['Radar']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
final_df

,x,y,B04B,B05B,B06B,B09B,B10B,B11B,B12B,B14B,...,SSHF,TCLW,TCW,TCWV,U250,U850,V250,V850,Radar,datetime
0,104.90,19.96,0.498362,0.352224,0.236776,255.426270,260.791107,279.255859,259.747589,281.535248,...,-137404.0,0.601746,35.615921,35.005508,25.895142,-4.906418,6.482254,5.172928,0.0,2019-04-01 08:00:00
1,104.94,19.96,0.498362,0.352224,0.236776,255.426270,260.791107,279.255859,259.747589,281.535248,...,-137404.0,0.601746,35.615921,35.005508,25.895142,-4.906418,6.482254,5.172928,0.0,2019-04-01 08:00:00
2,104.98,19.96,0.572723,0.384196,0.249166,255.300003,260.903687,280.626465,260.545990,283.248993,...,-137404.0,0.601746,35.615921,35.005508,25.895142,-4.906418,6.482254,5.172928,0.0,2019-04-01 08:00:00
3,104.86,19.92,0.532949,0.360718,0.238078,255.813766,260.796844,278.823669,259.354004,280.841156,...,-272124.0,0.550171,32.744827,32.179337,26.195923,-4.334152,6.599442,3.686600,0.0,2019-04-01 08:00:00
4,104.90,19.92,0.532949,0.360718,0.238078,255.813766,260.796844,278.823669,259.354004,280.841156,...,-137404.0,0.601746,35.615921,35.005508,25.895142,-4.906418,6.482254,5.172928,0.0,2019-04-01 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388488,105.58,18.60,0.112172,0.079569,0.043030,249.934280,258.553345,280.853149,262.417236,283.495514,...,27460.0,0.573608,50.220444,49.587631,-8.698212,-8.955505,8.334885,-2.598038,0.0,2020-10-31 10:00:00
388489,105.62,18.60,0.110219,0.078257,0.042379,250.080978,258.668915,280.853149,262.611176,283.495514,...,27460.0,0.573608,50.220444,49.587631,-8.698212,-8.955505,8.334885,-2.598038,0.0,2020-10-31 10:00:00
388490,105.66,18.60,0.110219,0.078257,0.042379,250.080978,258.668915,280.853149,262.611176,283.495514,...,79364.0,0.546570,54.480209,53.866928,-8.530243,-7.871521,7.733322,-1.887100,0.0,2020-10-31 10:00:00
388491,105.54,18.56,0.114797,0.080210,0.042369,249.934280,258.668915,281.032013,262.514191,283.572968,...,27460.0,0.573608,50.220444,49.587631,-8.698212,-8.955505,8.334885,-2.598038,0.0,2020-10-31 10:00:00


In [53]:
# Huấn luyện Random Forest (chỉ 1 lần, n_estimators = 100)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Dự đoán trên tập huấn luyện và tập kiểm tra
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Đánh giá mô hình
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("\nĐánh giá Random Forest:")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"R2 Score (Test): {test_r2:.4f}")


Đánh giá Random Forest:
Train RMSE: 0.6799
Test RMSE: 0.7969
R2 Score (Test): 0.6368


In [54]:
# Huấn luyện XGBoost
xgb_model = XGBRegressor(random_state=42, n_estimators=100)
xgb_train_losses = []
xgb_val_losses = []
eval_set = [(X_train, y_train), (X_test, y_test)]
xgb_model.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=False)
results = xgb_model.evals_result()
xgb_train_losses = results['validation_0']['rmse']
xgb_val_losses = results['validation_1']['rmse']

# Đánh giá XGBoost
y_pred_xgb = xgb_model.predict(X_test)
print("\nĐánh giá XGBoost:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_xgb)):.4f}")
print(f"R2 Score: {r2_score(y_test, y_pred_xgb):.4f}")

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(



Đánh giá XGBoost:
RMSE: 0.6802
R2 Score: 0.7354


In [55]:
# Vẽ biểu đồ loss
plt.figure(figsize=(10, 6))
plt.plot(rf_train_losses, label='RF Train Loss')
plt.plot(rf_val_losses, label='RF Val Loss')
plt.plot(xgb_train_losses, label='XGB Train Loss')
plt.plot(xgb_val_losses, label='XGB Val Loss')
plt.xlabel('Iteration')
plt.ylabel('MSE Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig(os.path.join(OUTPUT_PATH, 'loss_curve_nghean_filtered.png'))
plt.close()

In [56]:
# Vẽ bản đồ lượng mưa cho một thời điểm
# Chọn một thời điểm ngẫu nhiên từ final_df
sample_dt = final_df['datetime'].sample(1).iloc[0]
sample_df = final_df[final_df['datetime'] == sample_dt].copy()

# Thêm dự đoán vào sample_df cho các hàng trong tập kiểm tra
test_indices = X_test.index
sample_df['prediction'] = np.nan
for idx in test_indices:
    if idx in sample_df.index:
        sample_df.loc[idx, 'prediction'] = y_pred_xgb[list(test_indices).index(idx)]

# Tạo bản đồ nhiệt
ground_truth_map = np.zeros((HEIGHT, WIDTH))
prediction_map = np.zeros((HEIGHT, WIDTH))
for _, row in sample_df.iterrows():
    with rasterio.open(radar_files_dict[sample_dt]) as src:
        transform = src.transform
        row_idx, col_idx = rasterio.transform.rowcol(transform, row['x'], row['y'])
        if 0 <= row_idx < HEIGHT and 0 <= col_idx < WIDTH:
            ground_truth_map[row_idx, col_idx] = row['Radar']
            if not np.isnan(row['prediction']):
                prediction_map[row_idx, col_idx] = row['prediction']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': ccrs.PlateCarree()})
ax1.set_title(f'Ground Truth - {sample_dt}')
ax2.set_title(f'Prediction (XGB) - {sample_dt}')
for ax, data in [(ax1, ground_truth_map), (ax2, prediction_map)]:
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    im = ax.imshow(data, cmap='Blues', origin='upper', transform=ccrs.PlateCarree(),
                   extent=(sample_df['x'].min(), sample_df['x'].max(), sample_df['y'].min(), sample_df['y'].max()))
    plt.colorbar(im, ax=ax, label='Lượng mưa (mm/h)')
plt.savefig(os.path.join(OUTPUT_PATH, f'rainfall_map_{sample_dt.strftime("%Y%m%d%H%M")}.png'))
plt.close()

In [57]:
# Vẽ scatter plot so sánh toàn bộ ground truth và dự đoán
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred_xgb, alpha=0.5, label='Dữ liệu')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Đường y=x')
plt.xlabel('Ground Truth (Lượng mưa mm/h)')
plt.ylabel('Dự đoán (Lượng mưa mm/h)')
plt.title('So sánh Ground Truth và Dự đoán XGBoost')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(OUTPUT_PATH, 'ground_truth_vs_prediction_scatter.png'))
plt.close()